# Exploring NFL Data from SportsDataIO Discovery Lab

This notebook explores NFL data provided by SportsDataIO Discovery Lab. We'll analyze various aspects of the data, visualize trends, and document insights using markdown cells for clarity and reproducibility.

In [1]:
# %pip install -r ../../requirements.txt

In [1]:
import pandas as pd
import numpy as np
import os
import sys

project_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
print(f"Project Root: {project_root}")
print("Sys Path Before:", sys.path)
if project_root not in sys.path:
    print("Inserting project root to sys.path")
    sys.path.insert(0, project_root)

# Now import
from data_api import SportsDataIO, Yahoo, PFR
from utils import describe_endpoint
from yahoo_helpers import get_all_players, get_player_details, get_player_stats

from dotenv import load_dotenv
load_dotenv()

Project Root: c:\Users\bengu\Documents\Sports Analysis Project\clairvoyent-raven-sports-analysis\src
Sys Path Before: ['C:\\Users\\bengu\\AppData\\Local\\Programs\\Python\\Python310\\python310.zip', 'C:\\Users\\bengu\\AppData\\Local\\Programs\\Python\\Python310\\DLLs', 'C:\\Users\\bengu\\AppData\\Local\\Programs\\Python\\Python310\\lib', 'C:\\Users\\bengu\\AppData\\Local\\Programs\\Python\\Python310', 'c:\\Users\\bengu\\.virtualenvs\\cfeproj-oIABPDjj', '', 'c:\\Users\\bengu\\.virtualenvs\\cfeproj-oIABPDjj\\lib\\site-packages', 'c:\\Users\\bengu\\.virtualenvs\\cfeproj-oIABPDjj\\lib\\site-packages\\win32', 'c:\\Users\\bengu\\.virtualenvs\\cfeproj-oIABPDjj\\lib\\site-packages\\win32\\lib', 'c:\\Users\\bengu\\.virtualenvs\\cfeproj-oIABPDjj\\lib\\site-packages\\Pythonwin']
Inserting project root to sys.path


True

In [4]:
pfr = PFR()

df = pfr.get_player_stats("Trevor Lawrence", 2024)

/players/L/LawrTr00.htm


In [5]:
df

,2024
year_id,
age,25
team_name_abbr,JAX
comp_name_abbr,NFL
pos,QB
games,10
games_started,10
qb_rec,2-8-0
pass_cmp,172
pass_att,284


In [21]:
"""
Use the Yahoo API to survey and assess the NFL fantasy football data available.
"""
yahoo_api = Yahoo()

all_players = get_all_players()
print(f"Total players loaded: {len(all_players)}")

# Get details for a sample of players
sample_player_details = get_player_details(yahoo_api, all_players[:5])

player_ids = []
for player in sample_player_details:
    player_ids.append(player["player_id"])

sample_player_stats = get_player_stats(yahoo_api, [player_id for player_id in player_ids], req_type="week", week=2)

[2025-09-26 16:28:41,599 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2025-09-26 16:28:41,606 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 2973.213094472885
[2025-09-26 16:28:41,607 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID


Getting league key
League key: 461.l.242497
Total players loaded: 2158
Error fetching details for A'Shawn Robinson: b'<?xml version="1.0" encoding="UTF-8"?>\n<error xml:lang="en-us" yahoo:uri="http://fantasysports.yahooapis.com/fantasy/v2/league/461.l.242497/players;search=A&amp;#39;Shawn%20Robinson/stats?format=json" xmlns:yahoo="http://www.yahooapis.com/v1/base.rng" xmlns="http://www.yahooapis.com/v1/base.rng">\n <description>invalid param search (was modified by yiv_get_stripped )</description>\n <detail/>\n</error>'


In [22]:
sample_player_stats

,player_id,name,position_type,Pass Yds,Pass TD,Int,Rush Att,Rush Yds,Rush TD,Rec,Rec Yds,Rec TD,Ret TD,2-PT,Fum Lost,Targets,Fum Ret TD,total_points
0,31883,A.J. Brown,O,0.0,0.0,0.0,0.0,0.0,0.0,5.0,27.0,0.0,0.0,0.0,0.0,8.0,0.0,5.20
1,40242,A.T. Perry,O,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00


In [3]:
"""
    This script tests various endpoints of the Yahoo Sports API for NFL fantasy data.
    It retrieves and displays data such as player details, team rosters, league settings, and more.
"""

endpoints = [
    # GAME
    ("game_id", lambda api: api.game.get_game_id()),
    ("league_ids", lambda api: api.game.get_league_ids()),
    # LEAGUE
    ("current_week", lambda api: api.league.get_current_week()),
    ("draft_results", lambda api: api.league.get_draft_results()),
    ("end_week", lambda api: api.league.get_end_week()),
    ("player_details", lambda api: api.league.get_player_details(player="Josh Allen")),
    ("player_stats", lambda api, player_id: api.league.get_player_stats(player_ids=[player_id])),
    ("percent_owned", lambda api, player_id: api.league.get_percent_owned(player_ids=[player_id])),
    ("positions", lambda api: api.league.get_positions()),
    ("stat_categories", lambda api: api.league.get_stat_categories()),
]

# Initialize API
def main():
    api = Yahoo() 

    output_file = "../../data/Yahoo_data_description.txt"
    with open(output_file, "w", encoding="utf-8") as f:
        f.write("Yahoo Endpoints DataFrame Info\n")
        f.write("=" * 70 + "\n\n")

        for (name, func) in endpoints:
            try:
                if (name in ["player_stats", "percent_owned"]):
                    df = func(api, player_id=player_id)
                else:
                    df = func(api)
                if (name == "player_details") and (df is not None):
                    player_id = df.iloc[0]["player_id"]
                print(f"\nEndpoint: {name}")
                desc = describe_endpoint(name, df)
            except Exception as e:
                desc = f"Endpoint: {name}\n"
                desc += "-" * 50 + "\n"
                desc += f"Error calling endpoint: {e}\n\n"
            f.write(desc)
    print(f"Wrote descriptions to {output_file}")

if __name__ == "__main__":
    main()

[2025-09-25 12:30:13,380 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2025-09-25 12:30:13,380 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 454.56656861305237
[2025-09-25 12:30:13,380 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID


Getting league key
League key: 461.l.242497

Endpoint: game_id

Endpoint: league_ids

Endpoint: current_week

Endpoint: draft_results

Endpoint: end_week

Endpoint: player_details

Endpoint: player_stats

Endpoint: percent_owned

Endpoint: positions

Endpoint: stat_categories
Wrote descriptions to ../../data/Yahoo_data_description.txt


In [ ]:
"""
This script tests various endpoints of the SportsDataIO API and writes
the structure of the returned DataFrames to a text file.
"""

# List of methods to test
endpoints = [
    # CORE
    ("bye_weeks", lambda api: api.core.get_bye_weeks()),
    ("player_details", lambda api: api.core.get_player_details()),
    ("rookie_details", lambda api: api.core.get_rookie_details()),
    ("standings", lambda api: api.core.get_standings()),
    ("teams", lambda api: api.core.get_teams()),
    ("timeframes", lambda api: api.core.get_timeframes()),
    # FANTASY
    # SKIP ("dfs_slates_by_date", lambda api: api.fantasy.get_dfs_slates_by_date()),
    ("dfs_slates_by_week", lambda api: api.fantasy.get_dfs_slates_by_week()),
    ("defense_game_stats",   lambda api: api.fantasy.get_defense_game_stats()),
    ("defense_season_stats", lambda api: api.fantasy.get_defense_season_stats()),
    ("player_game_stats",   lambda api: api.fantasy.get_player_game_stats()),
    ("player_season_stats", lambda api: api.fantasy.get_player_season_stats()),
    ("projected_defense_game_stats", lambda api: api.fantasy.get_projected_defense_game_stats()),
    ("projected_defense_season_stats", lambda api: api.fantasy.get_projected_defense_season_stats()),
    ("projected_player_game_stats", lambda api: api.fantasy.get_projected_player_game_stats()),
    ("projected_player_season_stats", lambda api: api.fantasy.get_projected_player_season_stats()),
    # ODDS
    ("pregame_odds", lambda api: api.odds.get_pregame_odds()),
    # SKIP ("pregame_odds_line_movement", lambda api: api.odds.get_pregame_odds_line_movement(scoreid)),
    ("scores", lambda api: api.odds.get_scores()),
    ("stadiums", lambda api: api.odds.get_stadiums()),
    ("team_game_stats", lambda api: api.odds.get_team_game_stats()),
    ("team_season_stats", lambda api: api.odds.get_team_season_stats())
]

def main():
    api = SportsDataIO(os.getenv("SPORTS_DATA_IO_API_KEY")) 

    output_file = "../data/SportsDataIO_data_description.txt"
    with open(output_file, "w", encoding="utf-8") as f:
        f.write("SportsDataIO Endpoints DataFrame Info\n")
        f.write("=" * 70 + "\n\n")

        for (name, func) in endpoints:
            try:
                df = func(api)
                desc = describe_endpoint(name, df)
            except Exception as e:
                desc = f"Endpoint: {name}\n"
                desc += "-" * 50 + "\n"
                desc += f"Error calling endpoint: {e}\n\n"
            f.write(desc)
    print(f"Wrote descriptions to {output_file}")

if __name__ == "__main__":
    main()

Wrote descriptions to ../data/SportsDataIO_data_description.txt
